Cortus Un-Supervised Model Investigation
------------------------------------------------------------------------------------------------------------------
### Introduction
------------------------------------------------------------------------------------------------------------------
Author: Joshua Keegan
Process Memory unsupervised learning model to help determine points of interest 

------------------------------------------------------------------------------------------------------------------

In [1]:
import json
import numpy
import os
import pandas as pd
import sys
import sklearn

numpy.set_printoptions(threshold=sys.maxsize)

In [2]:
dataset = pd.read_csv('../Data/compiledCSV/datasetFinal-2.csv', sep=',', low_memory=False, index_col=[0])

# Cleaning the file
# Drop anything containing "memory_section_" as they are present in all dumps
# Also drop useless index column and replace nans with '-1'
dataset = dataset.fillna(0)
dataset = dataset.drop(['processName', 'baddr', 'bintype', 'class', 'dbg_file', 'compiled', 'compiler', 'file', 'humansz'], axis=1)

dataset

,processType,arch,binsz,bits,canary,retguard,crypto,endian,flags,havecode,...,count_2.0.50727.5483 (Win7SP1GDR.050727-5400),count_GetMetaDataInternalInterfaceFromPublic.1,count_MetaDataGetDispenser.1,count_WerReportSetUIOption.1,count_WerpSetReportFlags.1,count_WerpGetReportFlags.1,count_WerpGetReportSettings.1,count_NtSuspendProcess.1,count_NtResumeProcess.1,count_?Initialize@CCBase@DirectUI@@QEAAJIPEAVElement@2@PEAK@Z.1
0,benign,x86,236338886,64,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,malicious,x86,25153704,32,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,benign,x86,22073862,64,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,benign,x86,29220314,64,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,malicious,x86,28906153,32,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,benign,x86,13630366,64,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
622,benign,x86,12991481,64,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
623,benign,x86,12196647,64,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624,malicious,x86,20441804,32,False,False,False,little,0x00061826,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Find counts of level of permissions
dataset[dataset.filter(regex='_perms').columns] = dataset[dataset.filter(regex='_perms').columns].apply(lambda col:(pd.Categorical(col).codes))
dataset = pd.concat([dataset, pd.DataFrame(dataset[dataset.filter(regex='_perms').columns].stack().groupby(level=0).value_counts().unstack(fill_value=0).add_prefix("permissionCount_"))], axis=1)
dataset = dataset.drop(dataset.filter(regex='_perms').columns, axis=1)

# Grab count of interesting memory sections per process dump
dataset = dataset.drop(dataset.filter(regex='Memory_Section').columns, axis=1)
dataUniqueMemorySectionCount = dataset[dataset.filter(regex='_size').columns].gt(0).sum(axis=1)
dataset['uniqueMemorySectionCount'] = dataUniqueMemorySectionCount
dataset = dataset.drop(dataset.filter(regex='_size').columns, axis=1)

# Clean up string data into categorical data
dataset['processType'] = pd.Categorical(dataset['processType']).codes
dataset['arch'] = pd.Categorical(dataset['arch']).codes
dataset['bits'] = pd.Categorical(dataset['bits']).codes
dataset['canary'] = pd.Categorical(dataset['canary']).codes
dataset['retguard'] = pd.Categorical(dataset['retguard']).codes
dataset['crypto'] = pd.Categorical(dataset['crypto']).codes
dataset['endian'] = pd.Categorical(dataset['endian']).codes
dataset['flags'] = pd.Categorical(dataset['flags']).codes
dataset['havecode'] = pd.Categorical(dataset['havecode']).codes
dataset['machine'] = pd.Categorical(dataset['machine']).codes
dataset['static'] = pd.Categorical(dataset['static']).codes

dataset_true_labels = dataset['processType']
dataset = dataset.drop('processType', 1)

dataset = dataset[[c for c
            in list(dataset)
            if len(dataset[c].unique()) > 1]]

dataset = dataset.loc[(dataset.sum(axis=1) != 0), (dataset.sum(axis=0) != 0)]
dataset = dataset.loc[(dataset.sum(axis=1) != 1), (dataset.sum(axis=0) != 1)]
dataset = dataset.loc[(dataset.sum(axis=1) != 557), (dataset.sum(axis=0) != 557)]


non_floats = []
for col in dataset:
    if dataset[col].dtypes == "float64" or dataset[col].dtypes == "int64" or dataset[col].dtypes == "int" or dataset[col].dtypes == "float" or dataset[col].dtypes == "int8":
        non_floats.append(col)
    
dataset = dataset[non_floats]
dataset

/tmp/ipykernel_2764/832155557.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dataset = dataset.drop('processType', 1)
/tmp/ipykernel_2764/832155557.py:32: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataset = dataset.loc[(dataset.sum(axis=1) != 0), (dataset.sum(axis=0) != 0)]
/tmp/ipykernel_2764/832155557.py:33: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataset = dataset.loc[(dataset.sum(axis=1) != 1), (dataset.sum(axis=0) != 1)]
/tmp/ipykernel_2764/832155557.py:34: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=N

,binsz,bits,machine,static,streams,size,rip,format.1,imports,registers,...,count_Software\\Microsoft\\Internet Explorer\\BrowserEmulation,count_Software\\Microsoft\\Internet Explorer\\TabbedBrowsing,count_C:\\Users\\admin\\AppData\\Local\\Temp\\60F1B8980D109A556922D5000AE02010.exe,permissionCount_0,permissionCount_1,permissionCount_2,permissionCount_3,permissionCount_4,permissionCount_5,uniqueMemorySectionCount
0,236338886,1,0,0,14,236338886,4.288723e+09,13,40533,18,...,0.0,0.0,0.0,820,1328,499,265,344,0,1222
1,25153704,0,1,0,13,25153704,0.000000e+00,12,4282,11,...,0.0,0.0,0.0,2786,132,73,46,125,94,115
2,22073862,1,0,0,14,22073862,5.365450e+09,13,4982,18,...,0.0,0.0,0.0,2941,154,50,86,25,0,136
3,29220314,1,0,0,13,29220314,5.357191e+09,12,5967,18,...,0.0,0.0,0.0,2944,250,28,27,7,0,195
4,28906153,0,1,0,13,28906153,0.000000e+00,12,5417,11,...,0.0,0.0,0.0,2997,129,49,53,28,0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,13630366,1,0,0,13,13630366,5.357638e+09,12,3193,18,...,0.0,0.0,0.0,3011,204,17,24,0,0,155
622,12991481,1,0,0,13,12991481,5.366551e+09,12,3193,18,...,0.0,0.0,0.0,3011,204,17,24,0,0,155
623,12196647,1,0,0,13,12196647,5.365241e+09,12,3193,18,...,0.0,0.0,0.0,3011,204,17,24,0,0,155
624,20441804,0,1,0,13,20441804,0.000000e+00,12,3372,11,...,0.0,0.0,0.0,2999,120,47,61,29,0,93


In [ ]:
# k-means clustering
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score, accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import svm

def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_true_labels, test_size=0.3) # 70% training and 30% test


preprocessor = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=2, random_state=42)),
    ]
)

clusterer = Pipeline(
   [
       (
           "kmeans",
           svm.SVC(kernel='rbf')
           # KMeans(
           #     n_clusters=2,
           #     init="k-means++",
           #     n_init=5,
           #     max_iter=50,
           #     random_state=42,
           #     algorithm='full',
           #     tol=1
           # ),
       ),
   ]
)

pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
)

label_encoder = LabelEncoder()
true_labels = label_encoder.fit_transform(dataset_true_labels)
pipe.fit(X_train, y_train)

preprocessed_data = pipe["preprocessor"].transform(X_test)
predicted_labels = pipe["clusterer"]["kmeans"].predict(preprocessed_data)

print("Accuracy:", accuracy_score(y_test, predicted_labels))

fig, ax = plt.subplots()
# title for the plots
title = ('Decision surface of linear SVC ')
# Set-up grid for plotting.
X0, X1 = preprocessed_data[:, 0], preprocessed_data[:, 1]
xx, yy = make_meshgrid(X0, X1)

plot_contours(ax, pipe["clusterer"]["kmeans"], xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=y_test, cmap=plt.cm.coolwarm, s=2, edgecolors='k')
ax.set_ylabel('y label here')
ax.set_xlabel('x label here')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title(title)
ax.legend()
plt.show()


Accuracy: 0.8404255319148937
